# DocTable Examples: Insert and Delete
Here we show basics of inserting and deleting data into a doctable.

In [1]:
import random
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import doctable as dt

In [2]:
schema = (
    ('integer','id',dict(primary_key=True, autoincrement=True)),
    ('string','name', dict(nullable=False)),
    ('integer','age'),
    ('boolean', 'is_old'),
)

In [3]:
def make_rows(N=3):
    rows = list()
    for i in range(N):
        age = random.random() # number in [0,1]
        is_old = age > 0.5
        yield {'name':'user_'+str(i), 'age':age, 'is_old':is_old}
    return rows

# Basic Inserts
There are only two ways to insert: one at a time (pass single dict), or multiple at a time (pass sequence of dicts).

In [4]:
db = dt.DocTable(schema, verbose=True)
for row in make_rows():
    db.insert(row)
db.select_df(verbose=False)

DocTable2 Query: INSERT OR FAIL INTO _documents_ (name, age, is_old) VALUES (:name, :age, :is_old)
DocTable2 Query: INSERT OR FAIL INTO _documents_ (name, age, is_old) VALUES (:name, :age, :is_old)
DocTable2 Query: INSERT OR FAIL INTO _documents_ (name, age, is_old) VALUES (:name, :age, :is_old)


,id,name,age,is_old
0,1,user_0,0.293036,False
1,2,user_1,0.534513,True
2,3,user_2,0.025267,False


In [5]:
db = dt.DocTable(schema, verbose=True)
db.insert(list(make_rows()))
db.select_df(verbose=False)

,id,name,age,is_old
0,1,user_0,0.712753,True
1,2,user_1,0.782320,True
2,3,user_2,0.400103,False


## Deletes

In [6]:
# delete everything
db = dt.DocTable(schema, verbose=True)
db.insert(list(make_rows()))
db.delete()
db.select_df(verbose=False)

DocTable2 Query: DELETE FROM _documents_


""


In [7]:
# delete all entries where is_old is false
db = dt.DocTable(schema, verbose=True)
db.insert(list(make_rows()))
db.delete(where=~db['is_old'])
db.select_df(verbose=False)

DocTable2 Query: DELETE FROM _documents_ WHERE NOT _documents_.is_old


,id,name,age,is_old
0,1,user_0,0.813919,True


In [8]:
# use vacuum to free unused space now
db = dt.DocTable(schema, verbose=True)
db.insert(list(make_rows()))
db.delete(where=~db['is_old'], vacuum=True)
db.select_df(verbose=False)

DocTable2 Query: DELETE FROM _documents_ WHERE NOT _documents_.is_old
DocTable2 Query: VACUUM


,id,name,age,is_old
0,2,user_1,0.850569,True
1,3,user_2,0.657028,True
